## Setup and Imports

In [ ]:
!git clone https://github.com/bnnebedum/Flood-Mapping.git
%cd Flood-Mapping
!git log --oneline -5

In [ ]:
!pip install -r requirements.txt
import sys
sys.path.append('/content/Flood-Mapping')

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except:
    IN_COLAB = False

if IN_COLAB:
    import subprocess
    import sys
    
    packages = ["rasterio", "pyyaml", "tqdm"]
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    
    from google.colab import drive
    drive.mount('/content/drive')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from pathlib import Path
import logging

import sys
if IN_COLAB:
    project_paths = [
        '/content/drive/MyDrive/Flood-Mapping',
        '/content/Flood-Mapping',
        '/content/drive/MyDrive/Flood-Mapping/src'
    ]
    
    for path in project_paths:
        if os.path.exists(path):
            sys.path.append(path)
            print(f"Added to path: {path}")
            break
    else:
        print("Project not found in expected locations. Please upload the project to your Drive.")
        print("Expected location: /content/drive/MyDrive/Flood-Mapping/")

try:
    from src.utils.config import ConfigManager, ExperimentConfig, get_default_config
    from src.utils.colab_utils import setup_colab_environment
    from src.training.trainer import FloodSegmentationTrainer
    from src.evaluation.visualizer import FloodSegmentationVisualizer
    print("[SUCCESS] All project modules imported successfully")
except ImportError as e:
    print(f"Import error: {e}")
    print("Please ensure the project structure is uploaded to your Google Drive")
    print("Required structure: /content/drive/MyDrive/Flood-Mapping/src/...")
    raise

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

from src.utils.data_validation import DataStructureValidator, quick_data_check

# Quick check
data_ready = quick_data_check()

if not data_ready:
    validator = DataStructureValidator()
    validator.print_validation_report()
    
    raise ValueError("Data structure validation failed. Please check the issues above.")

setup_result = setup_colab_environment()

data_verification = setup_result['data_verification']
total_pairs = 0
for split, info in data_verification['splits'].items():
    pairs = info['matched_pairs']
    total_pairs += pairs
    print(f"{split}: {pairs} matched pairs")

print(f"Total dataset: {total_pairs} matched pairs")

if setup_result['metadata']:
    if 'channel_analysis' in setup_result['metadata']:
        channel_analysis = setup_result['metadata']['channel_analysis']
        for channel, stats in channel_analysis.items():
            print(f"{channel}: contrast={stats['avg_contrast']:.3f}, separability={stats['avg_separability']:.3f}")
    
    if 'normalization_stats' in setup_result['metadata']:
        norm_stats = setup_result['metadata']['normalization_stats']
        print("Channel means:", [f"{m:.1f}" for m in norm_stats['mean']])
        print("Channel stds:", [f"{s:.1f}" for s in norm_stats['std']])
else:
    print("WARNING: No preprocessing metadata found. Using default normalization.")


## Configuration

In [ ]:
config_paths = [
    "/content/drive/MyDrive/Flood-Mapping/configs/unet_config.yaml",
    "/content/Flood-Mapping/configs/unet_config.yaml"
]

config = None
for config_path in config_paths:
    if os.path.exists(config_path):
        config = ConfigManager.load_config(config_path)
        print(f"Configuration loaded from: {config_path}")
        break

if config is None:
    print("Configuration file not found. Creating default configuration...")
    # Create default configuration
    config = get_default_config("unet")
    
    # Ensure the drive path matches your structure
    config.data.drive_path = "/content/drive/MyDrive/Preprocessed-128"
    config.output_dir = "/content/drive/MyDrive/experiments"
    
    print("Using default configuration")

config.data.drive_path = "/content/drive/MyDrive/Preprocessed-128"
config.output_dir = "/content/drive/MyDrive/experiments"

print("\nCONFIGURATION SUMMARY")
print(f"Experiment: {config.experiment_name}")
print(f"Model: {config.model.name}")
print(f"Batch size: {config.data.batch_size}")
print(f"Epochs: {config.training.epochs}")
print(f"Learning rate: {config.training.initial_lr}")
print(f"Data path: {config.data.drive_path}")
print(f"Output path: {config.output_dir}")

if not os.path.exists(config.data.drive_path):
    raise FileNotFoundError(f"Data path not found: {config.data.drive_path}")
print(f"Data path verified: {config.data.drive_path}")

## Model training

In [ ]:
trainer = FloodSegmentationTrainer(config)

trainer.setup_datasets()

trainer.setup_model()

print(f"\nModel has {trainer.model.count_params():,} parameters")

print("If you see this it means you have the most recent commit")
training_history = trainer.train()

## Evaluation

In [ ]:
test_results = trainer.evaluate_test_set()

for metric, value in test_results.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.4f}")

## Visualization

In [ ]:
visualizer = FloodSegmentationVisualizer(config.evaluation)

viz_results = visualizer.generate_sample_visualizations(
    model=trainer.model,
    dataset=trainer.test_dataset,
    output_dir=trainer.experiment_dir,
    num_samples=20
)

print(f"Generated {viz_results['num_samples']} visualizations")
print(f"Saved to: {viz_results['output_directory']}")

## Analyze results

In [ ]:
history_file = trainer.experiment_dir / 'training_history.json'
with open(history_file, 'r') as f:
    history_data = json.load(f)

final_metrics = history_data['final_metrics']

print("Final Training Metrics:")
print(f"  Train Loss: {final_metrics['train_loss']:.4f}")
print(f"  Val Loss: {final_metrics['val_loss']:.4f}")
print(f"  Train DICE: {final_metrics['train_dice_coefficient']:.4f}")
print(f"  Val DICE: {final_metrics['val_dice_coefficient']:.4f}")
print(f"  Train Pixel Accuracy: {final_metrics['train_pixel_accuracy']:.4f}")
print(f"  Val Pixel Accuracy: {final_metrics['val_pixel_accuracy']:.4f}")

## Plot

In [ ]:
def plot_training_history(history_data):
    history = history_data['history']
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(history['loss'], label='Training Loss')
    axes[0, 0].plot(history['val_loss'], label='Validation Loss')
    axes[0, 0].set_title('Model Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    
    # DICE Coefficient
    axes[0, 1].plot(history['dice_coefficient'], label='Training DICE')
    axes[0, 1].plot(history['val_dice_coefficient'], label='Validation DICE')
    axes[0, 1].set_title('DICE Coefficient')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('DICE')
    axes[0, 1].legend()
    
    # Precision
    axes[1, 0].plot(history['precision'], label='Training Precision')
    axes[1, 0].plot(history['val_precision'], label='Validation Precision')
    axes[1, 0].set_title('Precision')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    
    # Recall
    axes[1, 1].plot(history['recall'], label='Training Recall')
    axes[1, 1].plot(history['val_recall'], label='Validation Recall')
    axes[1, 1].set_title('Recall')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    
    plt.tight_layout()
    
    plot_path = trainer.experiment_dir / 'training_history_plot.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"Training history plot saved to: {plot_path}")
    
    plt.show()

plot_training_history(history_data)

## Save and summarize

In [ ]:
trainer.save_model(config.training.save_weights_only)

print(f"\nExperiment completed successfully!")
print(f"Results saved to: {trainer.experiment_dir}")
print(f"Experiment summary: {trainer.experiment_dir / 'experiment_summary.json'}")

print("EXPERIMENT SUMMARY")
print(f"Experiment Name: {config.experiment_name}")
print(f"Model: {config.model.name}")
print(f"Dataset Size:")
for split, info in data_verification['splits'].items():
    print(f"  {split}: {info['matched_pairs']} pairs")
print(f"\nFinal Test Results:")
for metric, value in test_results.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.4f}")
print(f"\nTraining Time: {history_data.get('training_time_seconds', 0)/60:.1f} minutes")
print(f"Total Parameters: {trainer.model.count_params():,}")
print(f"Results Directory: {trainer.experiment_dir}")